## 1. 데이터 불러오기

'1_훼손 영역에 대한 검출 가능성.ipynb'에서 생성한 데이터를 사용.

In [1]:
import os
import cv2

DATA_PATH = "res/data/"
DATA_LIST_PATH = f"{DATA_PATH}.datalist"

# --------------------------------

# 1. 불러와야 할 데이터들의 파일명 수집
if not os.path.exists(DATA_PATH): raise Exception("No data.")

file_basename_list = []

with open(DATA_LIST_PATH, "r") as f:
    file_basename_list = f.readlines()
    file_basename_list = [file_basename.rstrip() for file_basename in file_basename_list]

# --------------------------------

# 2. 데이터를 불러오는 함수
def load(file_basename_list=file_basename_list):
    for file_basename in file_basename_list:
        file_name, file_extension = os.path.splitext(file_basename)

        raw          = cv2.imread(f"{DATA_PATH}{file_basename}",                          cv2.IMREAD_COLOR)
        mask         = cv2.imread(f"{DATA_PATH}{file_name}_mask{file_extension}",         cv2.IMREAD_GRAYSCALE)
        mask_colored = cv2.imread(f"{DATA_PATH}{file_name}_mask-colored{file_extension}", cv2.IMREAD_COLOR)

        if raw is None:
            raise Exception(f"Imread failed: {file_basename} from {file_basename_list}")

        yield file_basename, raw, mask, mask_colored

In [2]:

data_list = list(load())

import math

class CV2_Interface_HEPHEIR:
    def __init__(self, win_name, data_list=data_list):
        self.mx = 0
        self.my = 0

        self.win_name = win_name
        self.trackbar_name = 'Choose Image'

        self.file_basename = ''
        self.frame_raw = None
        self.frame_mask = None
        self.frame_mask_colored = None

        self.data_list = data_list

        cv2.namedWindow(self.win_name)
        cv2.setMouseCallback(self.win_name, self.onMouse)
        cv2.createTrackbar(self.trackbar_name, self.win_name, 0, len(self.data_list)-1, self.onTrackbarChange)

        self.updateDataset(self.data_list[0])

    # with 문 지원.
    def __enter__(self):
        return self
    
    def __exit__(self, type, value, traceback):
        cv2.destroyWindow(self.win_name)

    # --------------------------------

    def onMouse(self, event,x,y,flags,param):
        """마우스 포인터의 좌표 갱신"""
        if event == cv2.EVENT_MOUSEMOVE:
            self.mx = x
            self.my = y

    def onTrackbarChange(self, x):
        self.updateDataset(self.data_list[x])
    
    def updateDataset(self, data):
        file_basename, frame_raw, frame_mask, frame_mask_colored = data

        self.file_basename = file_basename
        self.frame_raw = frame_raw
        self.frame_mask = frame_mask
        self.frame_mask_colored = frame_mask_colored

    # --------------------------------

    def makePoints(self, n, cx, cy, min_point, max_point):
        """중심의 좌표(cx,cy)와 한 변의 길이(n)을 이용하여 정사각형의 두 꼭짓점의 좌표를 계산."""
        x0, y0 = math.floor(cx - n/2), math.floor(cy - n/2)
        x1, y1 = math.floor(cx + n/2), math.floor(cy + n/2)

        # 범위 밖 좌표를 방지하기 위한 보정
        dx, dy = 0, 0
        padding = 4

        minx, miny = min_point
        maxx, maxy = max_point
        
        if (x0 < minx): dx = minx -x0 + padding
        if (x1 > maxx): dx = maxx -x1 - padding

        if (y0 < miny): dy = miny -y0 + padding
        if (y1 > maxy): dy = maxy -y1 - padding

        if dx * dy == 0:
            return x0, y0, x1, y1
        else:
            return self.makePoints(n, cx+dx, cy+dy, min_point, max_point)

    # --------------------------------

    def updateFrame(self):
        pass

## 2. 선호/비선호 - 정상/비정상 영역 판단

다음과 같이 레이블을 부여한다.

* 비선호(정상): 0
* 선호(훼손): 1

### 2.1 아래의 프로그램을 통해 데이터를 수집한다.

* OpenCV에서 제공하는 기능을 이용:
    * 화면위에 마우스 커서를 올리면, 28x28크기의 정사각형 테두리가 나타난다.
    * 숫자키 0~9를 입력하면, 입력한 숫자가 클래스의 이름(레이블)이 되어 저장된다.

### 2.2 위에서 생성한 클래스의 서브클래스를 두 개 생성하여, 다음과 같은 용도로 사용한다.  
* 학습 데이터 생성 모델
* 데이터 예측 모델

In [3]:
# 위 코드에 이어서...

from src.dataset import save

class DATASET_CREATE(CV2_Interface_HEPHEIR):
    def __init__(self):
        super().__init__('create dataset')

    # @Override
    def updateFrame(self, key):
        h, w = self.frame_raw.shape[:2]

        frame_addWeighted = cv2.addWeighted(self.frame_raw, 0.75, self.frame_mask_colored, 0.75, 0)
        x0, y0, x1, y1 = self.makePoints(28, self.mx, self.my, (0, 0), (w, h))

        cv2.rectangle(frame_addWeighted, (x0,y0), (x1,y1), (0,0,255), 1)
        cv2.imshow(self.win_name, frame_addWeighted)

        if key in "0123456789":
            x, y = self.frame_raw[y0:y1, x0:x1], key
            save([x], [y])

In [4]:
# 위 코드에 이어서...

with DATASET_CREATE() as datasetCreate:
    while True:
        key = chr(cv2.waitKey(10) & 0xFF)

        if key == 'q': break
        else: datasetCreate.updateFrame(key)

### 2.3 앞서 구현한 모델들을 실행하여 결과를 확인한다.

In [5]:
# 위 코드에 이어서...

buildModel = False

if buildModel:
    import tensorflow as tf
    model = tf.keras.Sequential([
            tf.keras.layers.Conv2D(
                filters=32,
                kernel_size=(3,3),
                padding='same',
                activation='relu',
                input_shape=(28,28,3)),
            tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
            tf.keras.layers.Dropout(0.25),
            
            tf.keras.layers.Conv2D(
                filters=32,
                kernel_size=(3,3),
                padding='same',
                activation='relu'),
            tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
            tf.keras.layers.Dropout(0.25),

            tf.keras.layers.Flatten(),

            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dropout(0.2),

            tf.keras.layers.Dense(28, activation='relu'),
            tf.keras.layers.Dropout(0.2),

            tf.keras.layers.Dense(2, activation='softmax')
        ])

    model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

    # --------------------------------

    from src.dataset import load

    (x_train, y_train), (x_test, y_test) = load('res/data/classes')

    x_train, y_train = x_train / 255.0, y_train / 255.0
    x_test,  y_test  = x_test  / 255.0, y_test  / 255.0
    model.fit(x_train, y_train, epochs=16)
    model.evaluate(x_test, y_test)

    model.save('model/model_3_epoch=16')

In [6]:
# 위 코드에 이어서...

import tensorflow as tf
import numpy as np

class DATASET_PREDICT(CV2_Interface_HEPHEIR):
    def __init__(self):
        super().__init__('predict data')
        self.model = tf.keras.models.load_model('model/model_3_epoch=16')
    
    # @Override
    def updateFrame(self):
        h, w = self.frame_raw.shape[:2]
        x0, y0, x1, y1 = self.makePoints(28, self.mx, self.my, (0, 0), (w, h))

        frame_cut = self.frame_raw[y0:y1, x0:x1]
        
        predict = self.model.predict(np.expand_dims(frame_cut, axis=0))[0]
        predict = np.where(predict == max(predict))[0][0]

        frame_out = cv2.addWeighted(self.frame_raw, 0.75, self.frame_mask_colored, 0.75, 0)

        cv2.putText(frame_out, f"{predict}", (16,16), cv2.FONT_HERSHEY_PLAIN, 1.2, (0,0,255), lineType=cv2.LINE_AA)
        cv2.rectangle(frame_out, (x0,y0), (x1,y1), (0,0,255), 1)

        cv2.imshow(self.win_name, frame_out)

In [7]:
# 위 코드에 이어서...

with DATASET_PREDICT() as datasetPredict:

    while True:
        key = cv2.waitKey(10)

        if key == ord('q'): break
        else: datasetPredict.updateFrame()

ValueError: Error when checking input: expected conv2d_4_input to have shape (28, 28, 3) but got array with shape (28, 27, 3)